In [1]:
!pip install langchain-community
!pip install langchain_groq
!pip install PyPDF2
!pip install langchain_groq
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.

In [25]:
from langchain_groq import ChatGroq
import PyPDF2 as pdf
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [3]:
llm = ChatGroq(
    model_name = "llama-3.3-70b-versatile",
    groq_api_key = "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",
    temperature = 0
)

In [8]:
def input_pdf_text(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = pdf.PdfReader(file)
        text = ""
        for page in reader.pages:
            extracted = page.extract_text()
            if extracted:
                text += extracted
    return text

In [9]:
texts = input_pdf_text("/kaggle/input/ahkampdf/ahkam.pdf")

In [10]:
texts[:500]

'اقسام نمازها \n نمازهای واجب و مستحب  \nنمازهای واجب: \n      نمازهای یومی ه  \n      نماز طواف خانه ی خدا که پس از طواف خانه ی کعبه گزارده می شود. \n      نماز آیات که در هنگام پدید آمدن خورشید و ماه گرفتگی و زلزله و امثال آنها خوانده می شود. \n      نماز میت که بر مرده خوانده می شود. \n      نماز قضای پدر و مادر که بر پسر بزرگتر واجب است . \n      نمازی که به واسطه ی نذر و عهد و قسم یا به واسطه ی اجاره واجب شده است. \nنمازهای مستحب، مثل نافله های شبانه روزی. \nتوجه: \n نماز گویند و از میان نمازهای نافله،'

In [12]:
re_texts = re.sub("\s+", " ", texts)

In [13]:
re_texts[:500]

'اقسام نمازها نمازهای واجب و مستحب نمازهای واجب: نمازهای یومی ه نماز طواف خانه ی خدا که پس از طواف خانه ی کعبه گزارده می شود. نماز آیات که در هنگام پدید آمدن خورشید و ماه گرفتگی و زلزله و امثال آنها خوانده می شود. نماز میت که بر مرده خوانده می شود. نماز قضای پدر و مادر که بر پسر بزرگتر واجب است . نمازی که به واسطه ی نذر و عهد و قسم یا به واسطه ی اجاره واجب شده است. نمازهای مستحب، مثل نافله های شبانه روزی. توجه: نماز گویند و از میان نمازهای نافله، نافله »نافله« های مستحبی بسیار است و آنها راهای شب'

In [17]:
def get_chunks(texts):
    text_splitter = RecursiveCharacterTextSplitter(
        separators = ["\n\n", "\n", " "],
        chunk_size = 1500,
        chunk_overlap = 150
    )
    chunks = text_splitter.split_text(texts)
    return chunks

In [18]:
chunks = get_chunks(re_texts)

In [19]:
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 17


In [21]:
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

/tmp/ipykernel_36/4241498436.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
2025-08-15 10:19:10.324536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755253150.727077      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755253150.839772      36 cuda_blas.cc:1418] Unable to register cuBLAS f

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
vector_db = FAISS.from_texts(chunks, embedding = embeddings)

In [26]:
chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = vector_db.as_retriever(),
    input_key = "query"
)

In [27]:
response = chain("اقسام نمازهای واجب و مستحب را نام ببر")

/tmp/ipykernel_36/3383496542.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain("اقسام نمازهای واجب و مستحب را نام ببر")


In [28]:
response.keys()

dict_keys(['query', 'result'])

In [29]:
print(response["result"])

نمازهای واجب:
1. نمازهای یومی ه
2. نماز طواف خانه ی خدا
3. نماز آیات
4. نماز میت
5. نماز قضای پدر و مادر
6. نمازی که به واسطه ی نذر و عهد و قسم یا به واسطه ی اجاره واجب شده است.

نمازهای مستحب:
1. نافله های شبانه روزی
2. نافله های یومیه (مانند نافله ی ظهر، نافله ی عصر، نافله ی مغرب، نافله ی عشا، نافله ی صبح، نافله ی شب)


In [30]:
response = chain("نماز نافله چیست؟")
print(response["result"])

نماز نافله به نمازهای مستحبی گفته می‌شود که در کنار نمازهای واجب خوانده می‌شوند. نمازهای نافله می‌توانند در هر زمانی از روز یا شب خوانده شوند، اما معمولاً در زمان‌های خاصی مانند قبل یا بعد از نمازهای واجب خوانده می‌شوند. نمازهای نافله به عنوان یک عمل مستحب شمرده می‌شوند و خواندن آنها ثواب و پاداش دارد.


In [31]:
response = chain("علت اصلی سرطان چیست؟")
print(response["result"])

من نمی‌دانم که علت اصلی سرطان چیست.


In [32]:
response = chain("شرایط مکان نمازگزار چگونه باید باشد؟")
print(response["result"])

شرایط مکان نمازگزار به این شرح است:
۱. مباح باشد (یعنی غصبی نباشد).
۲. بی حرکت باشد.
۳. از جاهایی نباشد که توقف در آن حرام است.
۴. جلوتر از قبر پیغمبر(ص) و امام(ع) نایستد.
۵. محل سجده ی نمازگزار پاک باشد.
۶. موجب نجاست لباس و بدن نمازگزار نشود.
۷. میان زن و مرد در حال نماز بنا بر احتیاط واجب حداقل یک وجب فاصله باشد.
۸. مسطح باشد.


In [33]:
response = chain("شرایط نماز مسافرانی که با اتوبوس های بین شهری سفر می کنند چیست؟")
print(response["result"])

بر مسافرانی که با اتوبوس های بین شهری سفر می کنند واجب است که در صورت خوف از فوت نماز، از راننده بخواهند که اتوبوس را در مکان مناسبی متوقف نماید، و بر راننده اجابت درخواست آنان واجب است، و اگر به علت عذر مقبول و یا بدون دلیل از توقف اتوبوس خودداری کند، در این صورت، تکلیف مسافران اگر خوف فوت وقت داشته باشند، این است که نماز را در اتوبوس در همان حال حرکت بخوانند و تا آنجا که امکان دارد جهت قبله، قیام، رکوع و سجود را رعایت کنند.


In [34]:
response

{'query': 'شرایط نماز مسافرانی که با اتوبوس های بین شهری سفر می کنند چیست؟',
 'result': 'بر مسافرانی که با اتوبوس های بین شهری سفر می کنند واجب است که در صورت خوف از فوت نماز، از راننده بخواهند که اتوبوس را در مکان مناسبی متوقف نماید، و بر راننده اجابت درخواست آنان واجب است، و اگر به علت عذر مقبول و یا بدون دلیل از توقف اتوبوس خودداری کند، در این صورت، تکلیف مسافران اگر خوف فوت وقت داشته باشند، این است که نماز را در اتوبوس در همان حال حرکت بخوانند و تا آنجا که امکان دارد جهت قبله، قیام، رکوع و سجود را رعایت کنند.'}